In [46]:
def combine_rgb(red, green, blue):
    rgb = (red << 16) | (green << 8) | blue
    return rgb

def extract_rgb(rgb):
    red = (rgb >> 16) & 0xFF
    green = (rgb >> 8) & 0xFF
    blue = rgb & 0xFF
    return red, green, blue

In [47]:
from PIL import Image
import csv

# Load the temperature color map image
image = Image.open("../Data/Images/original/2020_LST.bmp")

# Define the temperature range and corresponding RGB values
temperature_range = (0, 50)  # Temperature range from 0 to 50 degrees Celsius
color_range = [(255, 0, 0), (255, 255, 0), (0, 255, 0)]  # RGB color gradient from red to yellow to green

# Create a CSV file to store the temperature data
csv_file = open("temperature_data.csv", "w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["X", "Y", "rgb"])

# Iterate over each pixel in the image and extract temperature data
pixels = image.load()
width, height = image.size
for x in range(width):
    for y in range(height):
        r, g, b = pixels[x, y]
        if r == 0 and g == 0 and b == 0:
            temperature = -1
        else:
            # Calculate the temperature based on the color gradient
            # progress = (r / 255)  # Calculate the progress within the gradient (0 to 1)
            # temperature = temperature_range[0] + (temperature_range[1] - temperature_range[0]) * progress
            temperature = combine_rgb(r, g, b)

        # Save the temperature data along with the pixel position to the CSV file
        csv_writer.writerow([x, y, int(temperature)])

# Close the CSV file
csv_file.close()


In [1]:
import numpy as np
import csv
from PIL import Image

# Read temperature data from CSV file
temperature_data = []
path='./final/2000_og.csv'
img_path='temperature_data.png'

with open(path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row
    for row in reader:
        x, y, temperature = map(int, row)
        temperature_data.append((x, y, temperature))

# Find min and max temperature values
temperatures = [temp for _, _, temp in temperature_data]

# find min temperaute but skip -1
min_temp = min([temp for temp in temperatures if temp != -1])
max_temp = max(temperatures)

# Create an empty image array
width = max(x for x, _, _ in temperature_data) + 1
height = max(y for _, y, _ in temperature_data) + 1
image_array = np.zeros((height, width, 3), dtype=np.uint8)

# Convert temperature values back to color and fill the image array
for x, y, temperature in temperature_data:
    if temperature == -1:
        color = (0,0,0)
    else:
        # Interpolate color based on temperature value
        # normalized_temp = (temperature - min_temp) / (max_temp - min_temp)
        # green = int(255 * (1 - normalized_temp))
        # red = int(255 * normalized_temp)
        # color = (red, green, 0)  # Set blue channel to 0
        color = extract_rgb(temperature)

    # Fill the corresponding pixel in the image array with the interpolated color
    image_array[y, x] = color

# Create and save the image
image = Image.fromarray(image_array)
image.save(img_path)

print("Image restored from temperature data and saved as temp_color_map_restored.jpg")

ValueError: too many values to unpack (expected 3)

In [4]:
import cv2
import numpy as np
import csv

# Initialize variables to store the maximum X and Y values
max_x = 0
max_y = 0

# Load data from the CSV file and find the maximum X and Y values
csv_filename = "./final/2000_og.csv"  # Replace with your CSV filename
with open(csv_filename, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)  # Skip the header row if it exists

    for row in csv_reader:
        x, y, *_ = map(int, row)
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y

# Add some padding to the maximum X and Y values (optional)
padding = 10
max_x += padding
max_y += padding

# Create an all-white image based on the calculated width and height
image = np.ones((max_y, max_x, 3), np.uint8) * 255

# Reload the CSV file and draw points on the image
with open(csv_filename, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)  # Skip the header row if it exists

    for row in csv_reader:
        x, y, r, g, b,_ = map(int, row)
        color = (b, g, r)  # OpenCV uses BGR format
        image[y, x] = color

# Save the image
output_filename = "output_image.png"  # Replace with your desired output image filename
cv2.imwrite(output_filename, image)

True